# Define functions
First, we need to define the functions that we need to use for indentifying the spots in the images. Press the **play** button to execute the chunk of code and if you are curious about what is going on behind the scenes, **show code** expands the full code.

The way to execute the code is to either press the **play** button or press **shift+enter**

In [3]:
import numpy as np
import pandas as pd
import quantagen.image_processing
import quantagen.core
import quantagen.utils

# Define path to images and the desired output location
In this step it is important that your drive has been mounted correctly since we need to locate where the files are stored. We will point to the program where to look for your images.

Also, we will extract the dimensions of your images for you the user to check if the dimensions extracted (C - channels; Z - Z-stacks; X & Y pixel dimension) are correct, and if they are specified in the right order (see below).

In [4]:
path = '/Users/chrislangseth/development/Quantagen/data/test_data/'
output_folder = '/Users/chrislangseth/test_output/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
files = quantagen.image_processing.get_image_dimensions(path)

 
Image 1, Z dimension seems to be (1): 5
Image 1, X dimension seems to be (2): 2048
Image 1, Y dimension seems to be (3): 2048
 
Image 2, Z dimension seems to be (1): 5
Image 2, X dimension seems to be (2): 2048
Image 2, Y dimension seems to be (3): 2048
 
Image 3, Z dimension seems to be (1): 5
Image 3, X dimension seems to be (2): 2048
Image 3, Y dimension seems to be (3): 2048
 
Image 4, Z dimension seems to be (1): 5
Image 4, X dimension seems to be (2): 2048
Image 4, Y dimension seems to be (3): 2048
 
Image 5, Z dimension seems to be (1): 5
Image 5, X dimension seems to be (2): 2048
Image 5, Y dimension seems to be (3): 2048
 
Image 6, Z dimension seems to be (1): 5
Image 6, X dimension seems to be (2): 2048
Image 6, Y dimension seems to be (3): 2048
 
Please proceed to the analysis only if dimensions are correct. Otherwise you need to change the parameters specified in index values in countagen_main, following the instructions in the IMPORTANT NOTE section


# Important note!
Now it is time to make sure that your images have the dimensions specified in the right order. Have a look at the output from the previous cell. Does this make sense?

If you think the order is not accurate (ie. channels and number of z-planes are inverted), you need to change the **numbering** of the following parameters in the next chunk of code, where the *countagen_main* function is applied.

1. channel_index = 0
2. z_stack_index = 1
3. x_index = 2
4. y_index = 3

As an example:
Imagine you took an image with 7 z-planes, in 3 colours, with a XY resolution of 1000x2000, you load it in the script and the output from the previous block of code says:

*Image A, C dimension seems to be (0): 7*

*Image A, Z dimension seems to be (1): 3*

*Image A, X dimension seems to be (2): 1000*

*Image A, Y dimension seems to be (3): 2000*

This is a sign that your C (channel) and Z dimensions have been misattributed. This sometimes can happen depending on the software you used to generate the images. In this specificic case your image's dimensions are specified in the order ZCXY, while this scripts wants the images to be specified in the order CZXY.

This problem can easily be fixed by re-specifying the sequence of dimensions in the following function. Keep in mind that python begins to count from 0, so the first dimension has position 0, the second has 1, etc....

In this specific case, where we want to convert a ZCXY image into a CZXY image, we just need to invert the numbering of the parameters:

channel_index = (we now should set it to 1)

z_stack_index = (we now should set it to 0)

This will shuffle the image dimension accordingly, and proceed to image processing.

**Because the script processes entire folders, it is important that the image orders are specified in the same way for all the files in the folder. If you have files with different dimension orders in a single folder, please move them to separate subfolders and process each subfolder independently.**


# Main function
Now it is time to run the main function in the detection.

## Parameters to play with:


1. **project:**: this specifies whether the images need to be projected (True) or if they have already been projected with an external software (False)
2. **threshold**: this is the most important parameter in the detection. Smaller threshold will allow more spots to be captured but you run the risk of capturing more noise.
3. **mult**: this is multiplication factor that we apply to the images for visualization purposes and **will not** have an effect on the output. Default here is 10.
4. **radius**: this is the top hat filter radius to use in the filtering of the images. Bigger values will allow bigger spots to be kept.

Dimensions order:  see the documentation of the previous block to understand how to manipulate these parameters according to your image type.

In [5]:
csv_list = []
samples=np.unique([el.split('_')[0] for el in files])
for file in samples:
    filtered=[el for el in files if file in el[0:len(file)]]
    print(filtered)
    channels=[el.split(file+'_')[1][:-4] for el in filtered]
    suffixes=[el.split(file+'_')[1] for el in filtered]
    suffixes.sort()
    print(file)
    image=quantagen.image_processing.group_channels(path,file,suffixes)
    print('Grouping done')
    concat_df,image_norm = quantagen.core.countagen_main(
        image = image,
        channel_index = 0,
        z_stack_index = 1,
        threshold = 1.4,
        run_rolling_ball=False,
        footprint_value = 2,
        max_radius = 10,
        min_radius = 3,
        plot_blobs = False,
        path_file=output_folder+file.split('.')[0],
        channels=channels
    )
    print('write output')
    concat_df['file'] = file
    concat_df=quantagen.utils.add_quality(concat_df,image_norm)
    
    concat_df['target'] = 'channel_' + concat_df['channel'].astype(str)
    concat_df.to_csv(output_folder+file.split('.')[0]+'.csv')
    metrics = quantagen.utils.extract_metrics(concat_df)
    metrics['file'] = file
    metrics.to_csv(output_folder+file.split('.')[0]+'_metrics.csv')
    csv_list.append(metrics)
summary_stats = pd.concat(csv_list)
summary_stats.to_csv(output_folder+'summary_stats.csv')

['15-a_RAW_ch02.tif', '15-a_RAW_ch01.tif', '15-a_RAW_ch00.tif']
15-a
Grouping done
0
(2048, 2048)
The number of spots detected in your image is: 513
1
(2048, 2048)
The number of spots detected in your image is: 87
2
(2048, 2048)
The number of spots detected in your image is: 1101
channel
2    1101
0     513
1      87
Name: count, dtype: int64
write output
0
1
2
['5-a_RAW_ch02.tif', '5-a_RAW_ch00.tif', '5-a_RAW_ch01.tif']
5-a


/Users/chrislangseth/development/Quantagen/quantagen/utils.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '501.689669594312' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  concat_df.iloc[num,concat_df.shape[1]-1]=quality_of_spot


Grouping done
0
(2048, 2048)
The number of spots detected in your image is: 394
1
(2048, 2048)
The number of spots detected in your image is: 166
2
(2048, 2048)
The number of spots detected in your image is: 1012
channel
2    1012
0     394
1     166
Name: count, dtype: int64
write output
0
1
2


/Users/chrislangseth/development/Quantagen/quantagen/utils.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '480.7579787234042' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  concat_df.iloc[num,concat_df.shape[1]-1]=quality_of_spot
